Nama : Ghilman Farhani Putra Aji

NIM : 20110006

# Import Library

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

import warnings
warnings.filterwarnings("ignore")

# Load Dataset

In [2]:
data=pd.read_csv("dataset_mentalhealth.csv")

In [3]:
data

,Question_ID,Questions,Jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...


# Pre-Processing

## Mengecek pertanyaan sama

In [4]:
duplicated_rows = data[data.duplicated(subset=['Questions'], keep=False)]

# Print the result
print(duplicated_rows)

    Question_ID                                          Questions  \
8       1155199  Jika saya terlibat dalam pengobatan, apa yang ...   
11      1259439  Jika saya terlibat dalam pengobatan, apa yang ...   

                                              Jawaban  
8   Karena memulai perawatan adalah langkah besar ...  
11  Perawatan awal adalah langkah besar bagi indiv...  


dikarenakan ada baris yang memiliki pertanyaan yang sama, akan tetapi memiliki jawaban yang berbeda, perlu menghapus salah satu dikarenakan akan membingungkan mesin

In [5]:
data.drop(12)

,Question_ID,Questions,Jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...


In [6]:
data=data.drop(11)

In [7]:
duplicated_rows = data[data.duplicated(subset=['Questions'], keep=False)]

# Print the result
print(duplicated_rows)

Empty DataFrame
Columns: [Question_ID, Questions, Jawaban]
Index: []


karena sudah tidak ada pertanyaan yang sama, maka bisa lanjut ke proses selanjutnya

## lowercasing

In [8]:
data['Questions'] = data['Questions'].str.lower()
data['Jawaban'] = data['Jawaban'].str.lower()

In [9]:
data

,Question_ID,Questions,Jawaban
0,1590140,apa yang dimaksud dengan penyakit mental?,penyakit mental adalah kondisi kesehatan yang ...
1,2110618,siapa yang terpengaruh oleh penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,apa penyebab penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,apa sajakah tanda-tanda peringatan penyakit me...,gejala gangguan kesehatan mental bervariasi te...
4,7657263,apakah penderita penyakit jiwa bisa sembuh?,"ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,bagaimana saya tahu kalau saya minum terlalu b...,menyortir jika anda minum terlalu banyak bisa ...
94,7807643,"jika ganja berbahaya, mengapa kita melegalkannya?","asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,bagaimana cara meyakinkan anak saya untuk tida...,anda tidak bisa. tetapi anda dapat mempengaruh...
96,6521784,apa status hukum (dan bukti) minyak cbd?,cannabidiol atau cbd adalah komponen ganja yan...


## Menghapus karakter non-alphanumerical

In [10]:
data['Questions'] = data['Questions'].str.replace('[^\w\s]', '')
data['Jawaban'] = data['Jawaban'].str.replace('[^\w\s]', '')

In [11]:
data

,Question_ID,Questions,Jawaban
0,1590140,apa yang dimaksud dengan penyakit mental?,penyakit mental adalah kondisi kesehatan yang ...
1,2110618,siapa yang terpengaruh oleh penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,apa penyebab penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,apa sajakah tanda-tanda peringatan penyakit me...,gejala gangguan kesehatan mental bervariasi te...
4,7657263,apakah penderita penyakit jiwa bisa sembuh?,"ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,bagaimana saya tahu kalau saya minum terlalu b...,menyortir jika anda minum terlalu banyak bisa ...
94,7807643,"jika ganja berbahaya, mengapa kita melegalkannya?","asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,bagaimana cara meyakinkan anak saya untuk tida...,anda tidak bisa. tetapi anda dapat mempengaruh...
96,6521784,apa status hukum (dan bukti) minyak cbd?,cannabidiol atau cbd adalah komponen ganja yan...


## Menghapus baris yang mengandung null

In [12]:
data.dropna(inplace=True)

In [13]:
data

,Question_ID,Questions,Jawaban
0,1590140,apa yang dimaksud dengan penyakit mental?,penyakit mental adalah kondisi kesehatan yang ...
1,2110618,siapa yang terpengaruh oleh penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,apa penyebab penyakit mental?,diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,apa sajakah tanda-tanda peringatan penyakit me...,gejala gangguan kesehatan mental bervariasi te...
4,7657263,apakah penderita penyakit jiwa bisa sembuh?,"ketika penyembuhan dari penyakit mental, ident..."
...,...,...,...
93,4373204,bagaimana saya tahu kalau saya minum terlalu b...,menyortir jika anda minum terlalu banyak bisa ...
94,7807643,"jika ganja berbahaya, mengapa kita melegalkannya?","asap ganja, misalnya, mengandung racun penyeba..."
95,4352464,bagaimana cara meyakinkan anak saya untuk tida...,anda tidak bisa. tetapi anda dapat mempengaruh...
96,6521784,apa status hukum (dan bukti) minyak cbd?,cannabidiol atau cbd adalah komponen ganja yan...


# Vectorisasi
menggunakan tf idf vectorizer

In [14]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(data['Questions'])

# Training Model

In [15]:
model = LinearSVC()
model.fit(X_train_vec, data['Questions'])

LinearSVC()

variabel x pada model diatas adalah hasil vector data pertanyaan dan variabel y nya adalah data pertanyaan

# Chatbot

In [16]:
print("Welcome to the Mental Health FAQ Chatbot!")
print("Ask a question or enter 'quit' to exit.")

while True:
    user_input = input("User: ")
    print("__________________________________________")
    
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break
    
    # Vectorize user input
    user_input_vec = vectorizer.transform([user_input.lower()])
    
    # Predict the intent
    predicted_intent = model.predict(user_input_vec)[0]
    
    # Implement response generation mechanism based on predicted intent
    response = data[data['Questions'] == predicted_intent]['Jawaban'].values[0] if predicted_intent in data['Questions'].values else "I'm sorry, I don't have a response for that question."
    
    print("Chatbot: " + response)
    print("******************************************")

Welcome to the Mental Health FAQ Chatbot!
Ask a question or enter 'quit' to exit.
User: apa itu vaping?
__________________________________________
Chatbot: "vaping" adalah istilah untuk menggunakan perangkat di mana cairan, sering dibumbui, diubah menjadi uap (karenanya, vaping) dan dihirup. perangkat vaping terdiri dari corong, tangki atau reservoir untuk menahan cairan, elemen pemanas dan baterai untuk memberi daya elemen. cairan dipanaskan untuk menjadi kabut, dihirup melalui mulut ke paru -paru di mana nikotin atau ganja memasuki aliran darah. setiap uap residu dihembuskan. perangkat vaping datang dalam beberapa jenis, bentuk, dan ukuran, disebut sebagai rokok elektronik, pena vape, vape, mod, tangki, atau e-hookah. cairan vaping (e-liquid, e-jus) terdiri dari pelarut, biasanya propilen glikol atau gliserol, rasa, dan nikotin atau ganja, meskipun mungkin tidak mengandung apa pun. di british columbia anda harus berusia 19 tahun untuk membeli produk vaping.
 jumlah anak muda yang mer

User: bagaimana cara menjaga hubungan sosial?
__________________________________________
Chatbot: banyak orang sendirian sekarang, tetapi kita tidak harus kesepian. kita semua dalam hal ini bersama.
 meskipun anda mungkin terpisah secara fisik dari teman, anggota keluarga, dan orang -orang terkasih lainnya, tidak pernah lebih penting untuk mempertahankan koneksi sosial tersebut. koneksi sosial adalah kesempatan untuk mencari dan berbagi dukungan, berbicara melalui perasaan yang sulit, berbagi tawa, tetap mutakhir dengan orang yang dicintai, dan saling membantu mengatasinya. pandemi ini sangat banyak untuk ditangani sendiri. sementara langkah-langkah seperti penurunan fisik dan isolasi diri diperlukan untuk memperlambat penyebaran virus, pemisahan fisik dapat memperkuat banyak emosi yang menantang seperti kesepian dan ketakutan.
 pikirkan berbagai cara untuk terhubung yang paling berarti bagi anda. misalnya, anda mungkin lebih suka obrolan video daripada panggilan telepon, atau anda mun

User: quit
__________________________________________
Chatbot: Goodbye!


# Kesimpulan
setelah menanyakan 10 pertanyaan yang mirip dengan dataset, didapatkan 10 jawaban yang sesuai